**Open file and make data copy**

In [1]:
import json

with open(f'E:/Peter/Scheme/datasets/22-04-247-00-ss.json', 'r+') as file:
    main_data = json.load(file)

In [27]:
data['images'][2]

{'id': 2,
 'dataset_id': 22,
 'category_ids': [],
 'path': '/datasets/scheme_copy/scheme_003.png',
 'width': 1240,
 'height': 1754,
 'file_name': 'scheme_003.png',
 'annotated': False,
 'annotating': [],
 'num_annotations': 0,
 'metadata': {},
 'deleted': False,
 'milliseconds': 0,
 'events': [],
 'regenerate_thumbnail': False}

In [4]:
import copy

data = copy.deepcopy(main_data)

**Useful to deal with subdirectories** 

(without COCO-json shift)

In [5]:
import os

def get_all_files_recursive(root, file_type=None, avoid_dirs=None):
    res = list()
    for file_name in os.listdir(root):
        full_file_path = os.path.join(root, file_name)
        if os.path.isfile(full_file_path):
            if file_type is None or file_name.split('.')[-1] == file_type:
                res.append(full_file_path)
        elif avoid_dirs is None or file_name not in avoid_dirs:
            res.extend(get_all_files_recursive(full_file_path, file_type, avoid_dirs))
    return res

In [18]:
images = get_all_files_recursive(f'E:/Peter/Scheme/scheme_copy', 'png', ['.exports', '.thumbnail'])

moved_images = list()
images_root = f'E:/Peter/Scheme/'
counter = 0

for file_name in images:
    if file_name.split('\\')[-1].lower() not in moved_images:
        moved_images.append(file_name.split('\\')[-1].lower())
        counter+=1
        new_file_name = 'scheme_'+str(counter)+'.png'
        print('{} -> {}'.format(file_name, new_file_name))
        shutil.copyfile(file_name, os.path.join(images_root, new_file_name))

E:/Peter/Scheme/scheme_copy\scheme_001.png -> scheme_1.png
E:/Peter/Scheme/scheme_copy\scheme_002.png -> scheme_2.png
E:/Peter/Scheme/scheme_copy\scheme_003.png -> scheme_3.png
E:/Peter/Scheme/scheme_copy\scheme_004.png -> scheme_4.png
E:/Peter/Scheme/scheme_copy\scheme_005.png -> scheme_5.png
E:/Peter/Scheme/scheme_copy\scheme_006.png -> scheme_6.png
E:/Peter/Scheme/scheme_copy\scheme_007.png -> scheme_7.png
E:/Peter/Scheme/scheme_copy\scheme_008.png -> scheme_8.png
E:/Peter/Scheme/scheme_copy\scheme_009.png -> scheme_9.png
E:/Peter/Scheme/scheme_copy\scheme_010.png -> scheme_10.png
E:/Peter/Scheme/scheme_copy\scheme_011.png -> scheme_11.png
E:/Peter/Scheme/scheme_copy\scheme_012.png -> scheme_12.png
E:/Peter/Scheme/scheme_copy\scheme_013.png -> scheme_13.png
E:/Peter/Scheme/scheme_copy\scheme_014.png -> scheme_14.png
E:/Peter/Scheme/scheme_copy\scheme_015.png -> scheme_15.png
E:/Peter/Scheme/scheme_copy\scheme_016.png -> scheme_16.png
E:/Peter/Scheme/scheme_copy\scheme_017.png -> sch

E:/Peter/Scheme/scheme_copy\scheme_145.png -> scheme_141.png
E:/Peter/Scheme/scheme_copy\scheme_146.png -> scheme_142.png
E:/Peter/Scheme/scheme_copy\scheme_147.png -> scheme_143.png
E:/Peter/Scheme/scheme_copy\scheme_148.png -> scheme_144.png
E:/Peter/Scheme/scheme_copy\scheme_149.png -> scheme_145.png
E:/Peter/Scheme/scheme_copy\scheme_150.png -> scheme_146.png
E:/Peter/Scheme/scheme_copy\scheme_151.png -> scheme_147.png
E:/Peter/Scheme/scheme_copy\scheme_152.png -> scheme_148.png
E:/Peter/Scheme/scheme_copy\scheme_153.png -> scheme_149.png
E:/Peter/Scheme/scheme_copy\scheme_154.png -> scheme_150.png
E:/Peter/Scheme/scheme_copy\scheme_155.png -> scheme_151.png
E:/Peter/Scheme/scheme_copy\scheme_156.png -> scheme_152.png
E:/Peter/Scheme/scheme_copy\scheme_157.png -> scheme_153.png
E:/Peter/Scheme/scheme_copy\scheme_158.png -> scheme_154.png
E:/Peter/Scheme/scheme_copy\scheme_159.png -> scheme_155.png
E:/Peter/Scheme/scheme_copy\scheme_160.png -> scheme_156.png
E:/Peter/Scheme/scheme_c

***Shifting using COCO-json data***

**Simple images filenames shift**

In [28]:
import shutil
import os
import copy

def make_image_filenames_shift(data, images_root, image_prefix, first_index, new_folder=None):
    data_copy =  copy.deepcopy(data)
    
    counter = first_index
    moved_images = list()
    digits_num = len(str(len(data_copy['images'])))

    for image in data_copy['images']:
        moved_images.append(image['file_name'].lower())
        orig_path = image['file_name']
        r = counter
        while len(str(r)) != digits_num:
            r = '0' + str(r)
        image['path'] = image_prefix+str(r)+'.png'
        image['file_name'] = image_prefix+str(r)+'.png'
        src = os.path.join(images_root, orig_path)
        if not os.path.exists(src):
            continue
        if new_folder is not None:
            dst_folder = os.path.join(images_root, new_folder)
        else:
            dst_folder = images_root
        dst = os.path.join(dst_folder, image['file_name'])
        print('\r', end='')
        print('{} -> {}'.format(src, dst), end='')
        if not os.path.exists(dst_folder):
            os.makedirs(dst_folder)
        shutil.copyfile(src, dst)
        counter+=1
        
    return data_copy

In [29]:
os.path.join('E:/Peter/Scheme/coco-annotator/datasets/scheme_copy', '/Peter/Scheme/scheme_copy')

'E:/Peter/Scheme/scheme_copy'

In [31]:
shifted_data = make_image_filenames_shift(data, 'E:/Peter/Scheme/coco-annotator/datasets/scheme_copy', 'scheme_', 1, '/Peter/Scheme/scheme_copy')

E:/Peter/Scheme/coco-annotator/datasets/scheme_copy\scheme_231_3.png -> E:/Peter/Scheme/scheme_copy\scheme_245.png

In [51]:
import json

with open(f'E:/Peter/Scheme/datasets/18-04-247-00-ss.json', 'w+') as file:
    json.dump(shifted_data, file)
file.close()

***Shifting using listdir***

**Make shift dictionary**

In [135]:
import os

image_nums = [int(file_name.split('_')[1].split('.')[0]) for file_name in os.listdir(r'E:/Peter/Scheme/datasets/scheme_copy')]
image_nums.sort()
shift = dict.fromkeys(image_nums, 0)
c = 1
for num in shift:
    r = c
    while len(str(r)) != 3:
        r = '0' + str(r)
    shift[num] = r
    c+=1

**Apply shift to images**

In [137]:
root = f'E:/Peter/Scheme/datasets/scheme_copy'

for num in shift:
    if num != shift[num]:
        file = os.path.join(root, f'scheme_{num}.png')
        new_file = os.path.join(root, f'scheme_{shift[num]}.png')
        os.rename(file, new_file)

In [149]:
root = r'/datasets/scheme_copy/'
for im in f_data['images']:
    im_num = im['file_name'].split('_')[1].split('.')[0]
    r = im_num
    while len(str(r)) != 3:
        r = '0' + str(r)
    new_name = f'scheme_{r}.png'
    im['file_name'] = new_name
    im['path'] = root + new_name

In [152]:
import json

with open(r'E:\Peter\Scheme\datasets\18-04-247-00-ss', 'r+') as file:
    f_data = json.load(file)

In [153]:
f_data

{'images': [{'id': 0,
   'dataset_id': 19,
   'category_ids': [],
   'path': '/datasets/scheme_copy/scheme_001.png',
   'width': 4967,
   'height': 3509,
   'file_name': 'scheme_001.png',
   'annotated': False,
   'annotating': [],
   'num_annotations': 0,
   'metadata': {},
   'deleted': False,
   'milliseconds': 0,
   'events': [],
   'regenerate_thumbnail': False},
  {'id': 1,
   'dataset_id': 19,
   'category_ids': [],
   'path': '/datasets/scheme_copy/scheme_010.png',
   'width': 3309,
   'height': 2339,
   'file_name': 'scheme_010.png',
   'annotated': False,
   'annotating': [],
   'num_annotations': 0,
   'metadata': {},
   'deleted': False,
   'milliseconds': 0,
   'events': [],
   'regenerate_thumbnail': False},
  {'id': 2,
   'dataset_id': 19,
   'category_ids': [],
   'path': '/datasets/scheme_copy/scheme_011.png',
   'width': 3309,
   'height': 2339,
   'file_name': 'scheme_011.png',
   'annotated': False,
   'annotating': [],
   'num_annotations': 0,
   'metadata': {},
  